In [1]:
import pickle

import geopandas as gpd
import osmnx as ox
from scipy.spatial import cKDTree
import networkx as nx

from my_paths import *
import Graph_functions

In [2]:
PATH_STRADE_CICLABILI_PICKLE_RAW = "../Data/Raw/Info_bici/strade_ciclabili.pickle"
PATH_STRADE_CICLABILI_PICKLE_STAGING = "../Data/Staging/Info_bici/strade_ciclabili.pickle"
PATH_STRADE_CICLABILI_GEOJSON_STAGING = "../Data/Staging/Info_bici/strade_ciclabili.geojson"

PATH_CICLABILI_PICKLE_STAGING = "../Data/Staging/Info_bici/ciclabile.pickle"
PATH_CICLABILI_GEOJSON_STAGING = "../Data/Staging/Info_bici/ciclabile.geojson"
PATH_GEOJSON_STRADE_E_CICLABILI_STAGING = "../Data/Staging/Analisi/strade_e_ciclabili.geojson"
PATH_GEOJSON_COMPLETO_STAGING = "../Data/Staging/Analisi/completo.geojson"

PATH_TEST_STAGING = "../Data/Staging/Analisi/"

## Creazione Strade ciclabili Graph (compreso ingestion da openstreet maps)

Ingestion da openstreet maps e salviamo in Raw (pickle)

In [3]:
place = "Milano, Italy"

bike_filter = (
    '["highway"]["bicycle"!~"no"]["bicycle"!~"dismount"]'
)
G_ox_strade = ox.graph_from_place(place, network_type='bike', custom_filter=bike_filter)
with open(PATH_STRADE_CICLABILI_PICKLE_RAW, "wb") as f:
    pickle.dump(G_ox_strade, f)

Carichiamo il file da Raw e modifichiamolo

In [9]:
with open(PATH_STRADE_CICLABILI_PICKLE_RAW, "rb") as f:
    G_strade = pickle.load(f)

Aggiungiamo un peso in base alla lunghezza della strada

In [10]:
for u, v, k, data in G_strade.edges(data=True, keys=True):
    data['weight'] = data['length'] * 2 # Per ora
    data["tipo"] = "Strade_ciclabili"
    data["artificiale"] = False

Aggiungiamo archi di congiunzione artificiali per connettere strade molto vicine tra di loro

In [11]:
G_strade = G_strade.to_undirected()
G_strade = Graph_functions.add_edge_near_nodes(G_strade,
                                               weight_moltiplicator=2,  # Per ora
                                               tipo = "Strade_ciclabili", artificiale = True)

Salva GeoJson & Pickle in staging

In [12]:
_, gdf_edges = ox.graph_to_gdfs(G_strade)

# Andrebbe pulito il geojson ma per ora lo usiamo solo per visualizzazione su kepler
gdf_edges.to_file(PATH_STRADE_CICLABILI_GEOJSON_STAGING, driver="GeoJSON")

with open(PATH_STRADE_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_strade, f)

## Creazione Ciclabili Graph

Modifico ad hoc il raw delle ciclabili. Se decideremo di utilizzare questo metodo dei graph, modificheremo l'ETL delle ciclabili in questo modo e aggiungeremo la conversione in graph e il salvataggio in pickle

In [3]:
# Carico Raw
gdf_ciclabili_raw = gpd.read_file(PATH_CICLABILI_RAW)
gdf_ciclabili_staging = gdf_ciclabili_raw.copy()

# Convertiamo Multilinestring in Linestring
gdf_ciclabili_staging["geometry"] = gdf_ciclabili_staging["geometry"].apply(lambda x: x.geoms[0])

# Altre operazioni di pulizia. Da considerare di rimuover il group_by dall'ETL delle piste
gdf_ciclabili_staging["fine_lavor"] = gdf_ciclabili_staging["fine_lavor"].astype(int)
gdf_ciclabili_staging["lunghezza"] = gdf_ciclabili_staging["lunghezza"].astype(int)
gdf_ciclabili_staging = gdf_ciclabili_staging.drop(["id_amat", "id_via","gerarchia"], axis=1)

G_ciclabili = Graph_functions.geojson_to_graph(gdf_ciclabili_staging, weight_moltiplicator=1, tipo = "Ciclabile", artificiale = False)
with open(PATH_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_ciclabili, f)

Anche qui aggiungiamo connessioni artificiali tra nodi vicini non connessi

In [4]:
# Carichiamo MultiDiGraph da file pickle
with open(PATH_CICLABILI_PICKLE_STAGING, "rb") as f:
    G_ciclabili = pickle.load(f)

# Aggiungiamo archi artificiali
G_ciclabili = G_ciclabili.to_undirected()
G_ciclabili = Graph_functions.add_edge_near_nodes(G_ciclabili, weight_moltiplicator = 1, tipo = "Ciclabili", artificiale = True)

# Salviamo pickle e geojson in staging
_, gdf_edges = ox.graph_to_gdfs(G_ciclabili)
gdf_edges.to_file(PATH_CICLABILI_GEOJSON_STAGING, driver="GeoJSON") # Anche questo andrebbe ripulito
with open(PATH_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_ciclabili, f)

## Proviamo a creare connessioni con networkx

Carichiamo da file i Graph (dai file pickle)

In [5]:
# Carichiamo Graph Ciclabili
with open(PATH_CICLABILI_PICKLE_STAGING, "rb") as f:
    G_ciclabili = pickle.load(f)

# Carichiamo Graph Strade
with open(PATH_STRADE_CICLABILI_PICKLE_STAGING, "rb") as f:
    G_strade = pickle.load(f)

for n in G_ciclabili.nodes:
    G_ciclabili.nodes[n]["origin"] = "strada"
for n in G_strade.nodes:
    G_strade.nodes[n]["origin"] = "pista"
G_compose = nx.compose(G_strade, G_ciclabili)
G_compose_merged = Graph_functions.add_edge_near_nodes(G_compose,
                                                       weight_moltiplicator = 1.5,
                                                       tipo = "Join_ciclabile_strade",
                                                       artificiale=True)

Uniamo in un unico graph e aggiungiamo le connessioni tra i due con lo stesso meccanismo usato prima

In [6]:
gdf_compose = ox.graph_to_gdfs(G_compose_merged, edges=True, nodes=False)
gdf_compose.to_file(PATH_GEOJSON_STRADE_E_CICLABILI_STAGING, driver="GeoJSON")

In [7]:
# Carichiamo Parchi GeoDataFrame
gdf_parchi = gpd.read_file(PATH_PARCHI_CLEAN)

G_compose_merged_parchi = Graph_functions.connect_poi_nodes_to_graph(G_compose_merged, gdf_parchi,
                                                                     weight_moltiplicator=1,
                                                                     tipo = "parchi",
                                                                     poi = True,
                                                                     artificial=True)

74


In [8]:
gdf_compose = ox.graph_to_gdfs(G_compose_merged_parchi, edges=True, nodes=False)
gdf_compose.to_file(PATH_GEOJSON_COMPLETO_STAGING, driver="GeoJSON")